In [1]:
import keras
from keras_unet.models import custom_unet

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
from PIL import Image

In [4]:
orig = glob.glob("data/imgs/*.png")
masks = glob.glob("data/masks/*.png")

In [5]:
imgs_list = []
masks_list = []
for image, mask in zip(orig, masks):
    imgs_list.append(np.array(Image.open(image).resize((512,512))))
    
    im = Image.open(mask).resize((512,512))
    masks_list.append(np.array(im))

imgs_np = np.asarray(imgs_list)
masks_np = np.asarray(masks_list)

In [6]:
print(imgs_np.max(), masks_np.max())

255 8


In [7]:
x = np.asarray(imgs_np, dtype=np.float32)/255
y = np.asarray(masks_np, dtype=np.float32)/255

In [8]:
print(x.max(), y.max())

1.0 0.03137255


In [9]:
print(x.shape, y.shape)

(1366, 512, 512, 3) (1366, 512, 512)


In [10]:
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 3)
print(x.shape, y.shape)

(1366, 512, 512, 3) (1366, 512, 512)


In [11]:
model = custom_unet(
    input_shape=(512, 512, 3),
    use_batch_norm=True,
    num_classes=8,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid')

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (956, 512, 512, 3)
y_train:  (956, 512, 512)
x_val:  (410, 512, 512, 3)
y_val:  (410, 512, 512)


In [13]:
# from keras_unet.utils import get_augmented

# train_gen = get_augmented(
#     x_train, y_train, batch_size=4,
#     data_gen_args = dict(
#         rotation_range=15.,
#         width_shift_range=0.05,
#         height_shift_range=0.05,
#         shear_range=50,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         vertical_flip=True,
#         fill_mode='constant'
#     ))

In [14]:
# sample_batch = next(train_gen)
# xx, yy = sample_batch
# print(xx.shape, yy.shape)

In [15]:
from keras.callbacks import ModelCheckpoint


model_filename = 'segm_model_v0.keras'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [16]:
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [17]:
model.compile(
    optimizer=Adam(), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    loss=['sparse_categorical_crossentropy'],
    #loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)

In [18]:
history = model.fit(
    x_train,
    y_train,
    steps_per_epoch=10,
    epochs=5,
    batch_size=8,
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint]
)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node mul defined at (most recent call last):
  File "C:\Users\Aditya\miniconda3\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\Aditya\miniconda3\lib\runpy.py", line 86, in _run_code

  File "c:\Users\adity\Desktop\Deep Forest\keras-unet\env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\adity\Desktop\Deep Forest\keras-unet\env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\adity\Desktop\Deep Forest\keras-unet\env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\adity\Desktop\Deep Forest\keras-unet\env\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Aditya\miniconda3\lib\asyncio\base_events.py", line 603, in run_forever

  File "C:\Users\Aditya\miniconda3\lib\asyncio\base_events.py", line 1909, in _run_once

  File "C:\Users\Aditya\miniconda3\lib\asyncio\even

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)